In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
import sys
import random
import pyautogui
from PyQt5 import QtWidgets, uic
import icon_rc_
import webbrowser

In [2]:
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import time
import csv #엑셀파일
import os
import win32com.client 
from openpyxl import Workbook
import xlwt
import math
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import openpyxl 
import matplotlib.pyplot as plt
import keras
from keras.models import load_model

Using TensorFlow backend.


In [3]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [],[] 
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back):]
        dataX.append(a)
        dataY.append(dataset[i:(i + look_back):])
    
    dataX=np.squeeze(dataX,axis=1) #차원 축소
    dataY=np.squeeze(dataY,axis=1) #차원 축소
    return np.array(dataX), np.array(dataY)

In [4]:
np.random.seed(7)

data1=pd.read_excel('x_data_final.xls') #x좌표 엑셀 데이터 불러옴 
data2=pd.read_excel('y_data_final.xls') #y좌표 엑셀 데이터 불러옴 
data3=pd.read_excel('speed_final.xls') #speed좌표 엑셀 데이터 불러옴 

pddata1=pd.DataFrame(data1) 
pddata1.head()
pddata2=pd.DataFrame(data2) 
pddata2.head()
pddata3=pd.DataFrame(data3) 
pddata3.head()

hap=[] #진짜 합
for j in range(len(pddata1)):
    x_data=np.array(pddata1.loc[j])
    y_data=np.array(pddata2.loc[j])
    s_data=np.array(pddata3.loc[j])

    hap1=[] #[x,y,속력] 데이터를 저장하고 있는 리스트
    
    for i in range(len(s_data)): 
        sum=[] #리스트 하나당 임시로 x,y,속력을 저장할 리스트 (for문 돌릴때마다 초기화)
        sum.append(x_data[i]) #x넣음
        sum.append(y_data[i]) #y넣음
        sum.append(s_data[i]) #속력넣음
        hap1.append(sum) #[x,y,속력] 하나의 리스트를 hap리스트에 넣음  

    j = j + 1
    
    hap.append(hap1)

n1 = np.array(hap)
print(n1.shape)

(1094, 99, 3)


In [5]:
num_sample = n1.shape[0] # 257개 Sample 데이터
num_sequence = n1.shape[1] # 99개 시계열 데이터
num_feature = n1.shape[2] #3개 Feature

dataset = n1

In [6]:
train_size = int(len(dataset) * 0.70) #학습 데이터 70%
test_size = len(dataset) - train_size #테스트 데이터 30%
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [7]:
look_back = 1 #이전 시간 단계 입력변수
#X는 지금 t 값이고, Y는 그 다음의 t+1 값임 (즉, X=t, Y=t+1)
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [8]:
from keras.layers import TimeDistributed
from keras.layers import InputLayer
from keras.layers import Reshape
from keras.layers import *
from keras.models import *
from keras.utils import *

model = Sequential() 
model.add(LSTM(32,return_sequences=True, input_shape=(num_sequence,num_feature)))  
model.add(Dense(3, activation='softmax')) # 3개의 예측
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

history = model.fit(trainX, trainY, epochs=10, batch_size=20, validation_split=0.2, verbose=1)

model.save('lstm_final.h5')

Train on 610 samples, validate on 153 samples
Epoch 1/10
610/610 [==============================] - 2s 4ms/step - loss: 1433.5840 - accuracy: 0.4458 - val_loss: 1358.2503 - val_accuracy: 0.7516
Epoch 2/10
610/610 [==============================] - 2s 3ms/step - loss: 1203.7598 - accuracy: 0.9794 - val_loss: 1202.5396 - val_accuracy: 1.0000
Epoch 3/10
610/610 [==============================] - 2s 3ms/step - loss: 1084.8052 - accuracy: 1.0000 - val_loss: 1092.3576 - val_accuracy: 1.0000
Epoch 4/10
610/610 [==============================] - 2s 3ms/step - loss: 988.2468 - accuracy: 1.0000 - val_loss: 1035.2482 - val_accuracy: 1.0000
Epoch 5/10
610/610 [==============================] - 2s 3ms/step - loss: 952.2644 - accuracy: 1.0000 - val_loss: 1011.3145 - val_accuracy: 1.0000
Epoch 6/10
610/610 [==============================] - 2s 4ms/step - loss: 933.0470 - accuracy: 1.0000 - val_loss: 996.0141 - val_accuracy: 1.0000
Epoch 7/10
610/610 [==============================] - 1s 2ms/step - lo

In [10]:
print("정확도: ",(model.evaluate(testX, testY)))

327/327 [==============================] - 0s 733us/step
정확도:  [990.2615253786793, 0.9970963597297668]
